In [ ]:
import numpy as np
from tabulate import tabulate

# Example input: Increase the number of points as needed
img1u = [859, 1147, 1794, 2400, 2790, 2344, 2008, 1361, 1400, 1500]
img1v = [2573, 2554, 2309, 1913, 1325, 1331, 1563, 2015, 2200, 2300]
img2u = [536, 784, 1460, 2211, 2726, 2154, 1778, 1113, 1200, 1300]
img2v = [2431, 2480, 2405, 2170, 1675, 1603, 1758, 2052, 2100, 2200]

n = len(img1u)
assert len(img1v) == n and len(img2u) == n and len(img2v) == n, "Mismatch in number of points."

# Initialize matrix A with the correct size
A = np.zeros((n, 9), dtype=np.float64)

# Populate matrix A
for i in range(n):
    A[i][0] = img1u[i] * img2u[i]
    A[i][1] = img1v[i] * img2u[i]
    A[i][2] = img2u[i]
    A[i][3] = img1u[i] * img2v[i]
    A[i][4] = img1v[i] * img2v[i]
    A[i][5] = img2v[i]
    A[i][6] = img1u[i]
    A[i][7] = img1v[i]
    A[i][8] = 1

# Display matrix A
print("Matrix A:")
print(tabulate(A))

# Perform SVD on matrix A
U, Sigma, VT = np.linalg.svd(A)

# Extract the fundamental matrix (f is the last row of VT)
f = VT[-1]
print("\nf (last row of VT):", f)

# Reshape into a 3x3 fundamental matrix
F = f.reshape(3, 3)
print("\nFundamental Matrix F:")
print(F)

# Force F to have rank 2 by setting the smallest singular value to 0
U, Sigma, VT = np.linalg.svd(F)
Sigma[2] = 0
F2 = np.dot(U, np.dot(np.diag(Sigma), VT))
print("\nRank-2 Forced Fundamental Matrix F2:")
print(F2)

# Use calibration matrix K (as in the original code)
K = np.array([
    [9.25692841e+03, 0, 4.58239711e+02],
    [0, 8.37883743e+04, 3.59148084e+02],
    [0, 0, 1]
])

KTK = np.matmul(K.T, K)
print("\nKT * K:")
print(KTK)

# Compute the essential matrix
E = np.matmul(KTK, F2)
print("\nEssential Matrix E:")
print(E)

# Verify ranks
F2Rank = np.linalg.matrix_rank(F2)
ERank = np.linalg.matrix_rank(E)
print("\nRanks:")
print("Rank of F2:", F2Rank)
print("Rank of E:", ERank)

# Extract translation vector t from E
U, Sigma, VT = np.linalg.svd(E)
t = U[:, 2]
print("\nTranslation Vector t:")
print(t)

# Construct skew-symmetric matrix for t
tX = np.array([
    [0, -t[2], t[1]],
    [t[2], 0, -t[0]],
    [-t[1], t[0], 0]
])
print("\n[t]x (Skew-Symmetric Matrix):")
print(tX)

# Compute rotation matrix
U, Sigma, VTt = np.linalg.svd(tX)
R = np.matmul(VTt, VT.T)
print("\nRotation Matrix R:")
print(R)

# Verify R is a valid rotation matrix
print("\nDeterminant of R:", np.linalg.det(R))
print("R * R^T:")
print(np.matmul(R, R.T))


In [ ]:
Xlist = []
Rnew = -R
tnew = (t + np.array([1, .1, 0])).reshape(3, 1)
print(R)
print(t)
Rt = np.hstack((Rnew, tnew))
print("Extrinsic Matrix:\n",Rt)

P = np.matmul(K, Rt)
print("Paramter Matrix:\n",P)

imagepoints = np.vstack((img2u, img2v, np.ones_like(img2u)))
print(imagepoints)

for i in range(imagepoints.shape[1]):
    row1 = img2u[i] * P[2, :] - P[0, :]
    row2 = img2v[i] * P[2, :] - P[1, :]
    A = np.array([row1, row2])
    U, Sigma, VT = np.linalg.svd(A)
    X = VT[-1]
    X = X / X[-1]

    Xlist.append(X[:-1])

print(Xlist)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

Xlist2 = [
    np.array([-0.02301545,  0.08762778, -0.02346701]),
    np.array([-0.05050731,  0.10871056, -0.00667381]),
    np.array([0.11992066, 0.16453816, 0.02863155]),
    np.array([0.19906501, 0.21458241, 0.06038518]),
    np.array([0.25110812, 0.24466381, 0.07581338]),
    np.array([0.19066146, 0.2146913 , 0.05395535]),
    np.array([0.1511105 , 0.19045173, 0.0387145 ]),
    np.array([0.08086257, 0.14068122, 0.00746419])
]
Xlist2array = np.array(Xlist2)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(Xlist2array[:, 0], Xlist2array[:, 1], Xlist2array[:, 2], c='b', marker='o')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.show()